In [1]:
# imports
import pandas as pd
from pyspark.sql import SparkSession
import os

new_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data_raw','RAW-Fitabase Data 4.12.16-5.12.16'))
os.chdir(new_dir)

print('Raw Data Directory 4/12 - 5/12 Fitabase:', os.getcwd())




Raw Data Directory 4/12 - 5/12 Fitabase: c:\Users\ashly\OneDrive\Documents\Education Material\ResearchProject\MaternalHealthResearch\predictive-model\data_raw\RAW-Fitabase Data 4.12.16-5.12.16


Phase 1 - Data Preprocessing 
1. Create Combination of Data for HeartRate to detect activity imbalances
2. Create ManualInput Dataset 
3. Clean all datasets


In [25]:
def CreateHeartRateDataset():
    
    # Test Spark Session for Merging Data
    spark = SparkSession.builder \
        .appName('Test DataFrame Merge') \
            .getOrCreate()
    
    # read the heart rate (HR) & mets dataset
    df_mets = spark.read.csv('minuteMETsNarrow_merged.csv', header=True, inferSchema=True)
    df_heartrate = spark.read.csv('heartrate_seconds_merged.csv', header=True, inferSchema=True)
    
    # df_mets.show()
    # df_heartrate.show()
    
    # Edit the Header Names 
    mets_column_names = ['id', 'timestamp', 'mets']
    heartrate_column_names = ['id', 'timestamp', 'bpm']
    df_mets = df_mets.toDF(*mets_column_names)
    df_heartrate = df_heartrate.toDF(*heartrate_column_names)
    
    df_merged = df_heartrate.join(df_mets, on=['id', 'timestamp'], how='inner')
    df_merged.show()
    
    df_merged.write.csv('test_spark.csv', header=True, mode='overwrite', sep=',')
    spark.stop()

In [26]:
CreateHeartRateDataset()

+----------+--------------------+---+----+
|        id|           timestamp|bpm|mets|
+----------+--------------------+---+----+
|2022484408|4/12/2016 10:04:0...|104|  62|
|2022484408|4/12/2016 10:11:0...|119|  70|
|2022484408|4/12/2016 10:19:0...|116|  72|
|2022484408|4/12/2016 10:20:0...|117|  72|
|2022484408|4/12/2016 10:23:0...|128|  82|
|2022484408|4/12/2016 10:29:0...|114|  72|
|2022484408|4/12/2016 11:01:0...| 73|  11|
|2022484408|4/12/2016 11:08:0...| 64|  11|
|2022484408|4/12/2016 11:14:0...| 67|  11|
|2022484408|4/12/2016 11:24:0...| 65|  10|
|2022484408|4/12/2016 11:26:0...| 64|  10|
|2022484408|4/12/2016 11:33:0...| 70|  24|
|2022484408|4/12/2016 11:40:0...| 70|  34|
|2022484408|4/12/2016 11:41:0...| 80|  30|
|2022484408|4/12/2016 11:44:0...| 57|  10|
|2022484408|4/12/2016 11:45:0...| 57|  10|
|2022484408|4/12/2016 11:46:0...| 60|  10|
|2022484408|4/12/2016 11:47:0...| 68|  10|
|2022484408|4/12/2016 11:56:0...| 63|  11|
|2022484408|4/12/2016 11:58:0...| 62|  11|
+----------

Py4JJavaError: An error occurred while calling o235.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: